In [2]:
import sympy as sp
import numpy as np
from scipy.integrate import solve_ivp

%matplotlib ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sympy import symbols, pi, sqrt, atan2, cos, sin
from sympy.diffgeom import Manifold, Patch, CoordSystem, metric_to_Christoffel_2nd

# `SymPy`'s Differential Geometry Module

`SymPy`'s differential geometry package provides useful tools for General Relativity.  The constructs we will be making use of are:
 * `Manifold`: represents a mathematical [manifold](https://en.wikipedia.org/wiki/Manifold) -- a topological space that locally resembles Euclidean space near each point -- which will pragmatically serve as a container for `patch`es we will define on the manifold.
 * `Patch`: represents a coordinate patch ('patch' for short) on a manifold, where coordinates can be defined.  This permits parameterization of any point on the patch in terms of the coordinates, and will serve as the container for coordinate system charts.
 * `CoordSystem`: a [coordinate system](https://en.wikipedia.org/wiki/Coordinate_system) that uses one or more coordinates to uniquely determine the position of the points or other geometric elements on a manifold. This object contains the logic for coordinate transformations.

## Example: Cartesian and polar coordinates in 2-D Euclidean space

First define our manifold and patch.

Now we create two coordinate systems with a defined transformation between them.

We can not define a point in one coordinate system (e.g., polar) and use that coordinate system's transform method to calculate the coordinates in another coordinate system (as long as the transformation is available in the `CoordSystem`'s relations.

We can also define a `Point` object in a chosen coordinate system, and compute coordinates in another.

Given a point, basis _scalar_ fields return the value of a particular coordinate.

Basis _vector_ fields provide unit vectors along a coordinate line.

# Wormhole Geometry

We've already used EinsteinPy to compute Christofell symbols for the metric

$$
ds^2 = -dt^2 + dr^2 + (b^2 + r^2)(d\theta^2 + \sin^2\theta d\phi^2),
$$

but let's now use `SymPy` directly.

In [11]:
from sympy.diffgeom import TensorProduct as TP

First we'll build up the line element.

In [ ]:
# Create a 4-D manifold.

# Create a patch.

# Constant symbols

# Coordinate system

# Get the coordinate functions

# Get the base one forms (differentials)

# Auxiliary terms for the line element.

# Build the line element

Are we in a coordinate basis? (TODO: I DONT THINK THIS CUTS IT)

In [ ]:
g_αβ = []

Now we can compute Christoffel symbols.

Now let's solve the geodesic equation (TODO: DISCUSS COORD BASIS)

$$
\frac{d^2 x^\alpha}{d\tau^2} = -\Gamma^\alpha_{\beta\gamma}\frac{dx^\beta}{d\tau}\frac{dx^\gamma}{d\tau}
$$

$$
\frac{d u^\alpha}{d\tau} = -\Gamma^\alpha_{\beta\gamma}u^\beta u^\gamma
$$


Let's work in the equatorial plane ($\theta=\pi/2$) without loss of generality (due to spherical symmetry).  We'll also set $b=1$, which is like working with a dimensionless coordinate radius $r'=r/b$.

Numerically speaking, we need to solve an initial value problem for a system of ordinary differential equations (ODEs).  We'll use `scipy`'s `solve_ivp()`.

In [ ]:
solve_ivp?

In [17]:
def F(t, y):
    """The system of ODEs to solve."""

def construct_four_velocity(uⁱ, metric=ds2, coords=SphCoords, subs=subs):
    """Given a three-velocity and metric construct the four-velocity"""

def eval_four_vector_field(u, x, coords=SphCoords):

Radial infall

In [18]:
R = 5       # initial coordinate radius distance
U = -.8     # initial radial velocity

# time steps to solve for the trajectory
T = np.linspace(0, 2*R/np.abs(U), 100)

In [22]:
def plot_double_polar(y, fig=None):
    if fig == None:
        fig = plt.figure(figsize=(8, 4))
    tmin, tmax = min(y[0]), max(y[0])

    in_pos_plane = y[1] > 0
    in_neg_plane = ~in_pos_plane

    ax = fig.add_subplot(121, projection='polar')
    ax.scatter(y[3][in_pos_plane], y[1][in_pos_plane], c=y[0][in_pos_plane], vmin=tmin, vmax=tmax)
    ax.grid(True)

    ax = fig.add_subplot(122, projection='polar')
    ax.scatter(y[3][in_neg_plane], abs(y[1][in_neg_plane]), c=y[0][in_neg_plane], vmin=tmin, vmax=tmax)
    ax.grid(True)
    ax.set_yticklabels(['-'+lab.get_text() for lab in ax.get_yticklabels()])
    return fig

Recall our work on the embedding diagram.  Working with cylindrical coordinates, we derived the 2-D surface in flat 3-D space to represent this line element's curvature:

$$
z(r, b) = b\, \text{sinh}^{-1}\left(\frac{r}{b}\right) \\
\rho(z) = b\, \text{cosh}\left(\frac{z}{b}\right)
$$

In [24]:
def get_z(r, b=1):

def ρ(z, b=1):

Let's construct a grid for plotting in 3-D.

Now let's plot the trajectory of our infalling particle on the embedding diagram.

In [26]:
def plot_trajectory_on_embedding(y=None, fig=None, ngrid=100, zmax=3):
    if fig is None:
        fig = plt.figure()
    zs = np.linspace(-1*zmax, zmax, ngrid)
    ρs = ρ(zs)

    ϕs, zs = np.meshgrid(np.linspace(0, 2*np.pi, ngrid), zs)
    ρs = ρ(zs)

    xs = (ρs*np.cos(ϕs)).flatten()
    ys = (ρs*np.sin(ϕs)).flatten()
    zs = zs.flatten()

    ax = fig.add_subplot(projection='3d')

    ax.plot(xs, ys, zs, linewidth=0.2)

    if y is not None:
        zs = get_z(sol.y[1])
        ρs = ρ(zs)
        ϕs = sol.y[3]
        xs = ρs*np.cos(ϕs)
        ys = ρs*np.sin(ϕs)
        ax.plot(xs, ys, zs, linewidth=1, c='r')
    return fig

Orbit?